In [ ]:
!pip install SimpleITK
!pip install skan #older version
!pip install tifffile

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
import SimpleITK as sitk
from skimage.morphology import skeletonize, thin, medial_axis
from skan import skeleton_to_csgraph, Skeleton
from skan import summarize
from skan import draw
import tifffile as tiff

import matplotlib.pyplot as plt
import numpy as np
import cv2
import plotly.express as px

In [8]:
def load_img_from_tiff(path2img):
    """
    Parameters
    ----------
    path2img: str 
        path to image Tiff file
    
    Returns
    -------
    img_array: np.array
        image data in numpy format
    """
    img = sitk.ReadImage(path2img)
    img_array = sitk.GetArrayFromImage(img)
    return(img_array)

Skeletonize single frame

In [17]:
single_thresh = load_img_from_tiff("/content/drive/MyDrive/mydata/sea_urchin_data/Test n.1")

# single_thresh = cv2.resize(single_thresh, dsize=(256,256), interpolation=cv2.INTER_CUBIC)

single_thresh = single_thresh.astype('float64')
single_thresh = (single_thresh - np.min(single_thresh))/np.ptp(single_thresh)
skimage_skeleton, dist = medial_axis(single_thresh, return_distance=True)
skan_skeleton = Skeleton(skimage_skeleton)
df=summarize(skan_skeleton)
# df.drop(df[df["branch-type"] < 2].index, inplace=True)

In [18]:
#rename columns:
df = df.rename(columns={'image-coord-src-0': 'src-y', 
                        'image-coord-src-1': 'src-x',
                        'image-coord-dst-0': 'dst-y',
                        'image-coord-dst-1': 'dst-x'})

df.drop(['mean-pixel-value',
         'stdev-pixel-value',
         'coord-src-0',
         'coord-src-1',
         'coord-dst-0',
         'coord-dst-1'], axis=1, inplace=True)
df.head()
df.to_csv("single_summary.csv")


In [ ]:
plt.rcParams["figure.figsize"] = (10,10)
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (15,15)
# draw.overlay_skeleton_2d(single_thresh, skimage_skeleton, dilate=1, axes=ax);
plt.imshow(single_thresh+skimage_skeleton, cmap='jet')
# plt.axis("off")
plt.title("Overlayed Skeleton")

In [ ]:
#overlay nodes 
plt.rcParams["figure.figsize"] = (10,10)
res_st = cv2.resize(single_thresh, dsize=(256,256), interpolation=cv2.INTER_CUBIC)
xs=np.array(df['src-x'])
ys=np.array(df['src-y'])
node_image = np.zeros([305,305], dtype=np.uint8)
# print(node_image.shape)
node_image[ys.astype(np.uint16), xs.astype(np.uint16)] = 4.
plt.imshow(node_image+skimage_skeleton+single_thresh)
plt.title("Overlayed Skeleton and Nodes")
# plt.scatter(ys,xs)

In [ ]:
px.imshow(dist, color_continuous_scale ="viridis")

Skeletonize multiple Z slices



In [ ]:
path = "/content/drive/MyDrive/mydata/sea_urchin_data/3D/Galleria Piastra Modello 1 100x100 Echi1-10x.tif"
data = load_img_from_tiff(path)
for i in range(len(data)):
    data[i] = data[i]*(255.0/data[i].max())

In [ ]:
blurred_data = np.zeros_like(data)
for i in range(len(blurred_data)):
    blurred_data[i] = cv2.GaussianBlur(data[i], (3,3),3)

thresholded_data = np.zeros_like(data)
for i in range(len(thresholded_data)):
    bin, thresholded_data[i] = cv2.threshold(blurred_data[i],175, 255, cv2.THRESH_BINARY)

medial_axis_skel = np.zeros_like(data)
for i in range(len(thresholded_data)):
    # skel, distance = medial_axis(thresholded_data[i], return_distance=True)
    # dist_on_skel = distance * skel
    # medial_axis_skel[i]=skel
    skel = skeletonize(thresholded_data[i]//255)
    medial_axis_skel[i]=skel

In [ ]:
skan_skel = Skeleton(medial_axis_skel)
df=summarize(skan_skel)
df.to_csv("z_slices_summary.csv")
#drop brances with length < 2
# df.drop(df[df["branch-distance"] < 2].index, inplace=True)
df